In [5]:
import os
%pwd

'd:\\Python Hub\\MLOPS\\End-to-End-ML-Approach-to-Wine-Quality-Prediction-with-MLflow\\research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'd:\\Python Hub\\MLOPS\\End-to-End-ML-Approach-to-Wine-Quality-Prediction-with-MLflow'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path 
    train_data_path: Path
    test_data_path: Path
    model_name: str 
    alpha: float 
    l1_ratio: float 
    target_column: str 

In [9]:
from MLproject.constants import * 
from MLproject.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer 
        params = self.params.ElasticNet 
        schema = self.schema.TARGET_COLUMN 

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            alpha= params.alpha,
            l1_ratio= params.l1_ratio,
            target_column= schema.name 
        )

        return model_trainer_config
    

In [12]:
import pandas as pd 
import numpy as np 
import os 
from MLproject import logger 
from sklearn.linear_model import ElasticNet
import joblib

In [16]:
class ModelTrainer:
    def __init__(self, config : ModelTrainerConfig ):
        self.config = config 

    def train(self):
        train_data = np.load(self.config.train_data_path)
        test_data = np.load(self.config.test_data_path)

        logger.info("Train and Test data loaded successfully")

        logger.info("Separate features (X) and target variable (y)")
        X_train = train_data[:, :-1]
        y_train = train_data[:, -1]

        X_test = test_data[:, :-1]
        y_test = test_data[:, -1]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(X_train, y_train)
        logger.info("Train the model on the training data")

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name)) 
        logger.info("Save the trained model to a file using joblib")      


In [17]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.error(f"Error during model training: {e}")
    raise e     

[ 2023-12-28 12:01:31,993:  INFO:  common: 33:  YAML file 'config\config.yaml' loaded successfully]
[ 2023-12-28 12:01:32,008:  INFO:  common: 33:  YAML file 'schema.yaml' loaded successfully]
[ 2023-12-28 12:01:32,012:  INFO:  common: 33:  YAML file 'params.yaml' loaded successfully]
[ 2023-12-28 12:01:32,014:  INFO:  common: 53:  Created directory at : artifacts]
[ 2023-12-28 12:01:32,017:  INFO:  common: 53:  Created directory at : artifacts/model_trainer]
[ 2023-12-28 12:01:32,023:  INFO:  2586983220: 9:  Train and Test data loaded successfully]
[ 2023-12-28 12:01:32,025:  INFO:  2586983220: 11:  Separate features (X) and target variable (y)]
[ 2023-12-28 12:01:32,033:  INFO:  2586983220: 20:  Train the model on the training data]
[ 2023-12-28 12:01:32,036:  INFO:  2586983220: 23:  Save the trained model to a file using joblib]
